## Compare runs, choose the best one, and deploy it

#### Run hyperparameter optimization

#### Compare runs

#### Deploy the best run and register the model

#### Deploy the model to the REST API

#### Build an image for the deployment

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)


In [3]:
# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)

In [4]:
signature = infer_signature(train_x, train_y)

In [5]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [6]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result


In [7]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [9]:
mlflow.set_tracking_uri(" http://127.0.0.1:5000")

In [10]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2024/10/23 00:39:43 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 18s 404ms/step - loss: 35.2453 - root_mean_squared_error: 5.8840
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - loss: 34.6237 - root_mean_squared_error: 5.8840
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.4843 - root_mean_squared_error: 5.8722
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.5409 - root_mean_squared_error: 5.8770
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 34.4627 - root_mean_squared_error: 5.8704
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.5522 - root_mean_squared_error: 5.8704
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 34.3448 - root_mean_squared_error: 5.8603
 8/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.1987 - root_mean_squared_error: 5.8477   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 33.0131 - root_mean_squared_error: 5.7455 - val_loss: 31.6491 - val_root_mean_squared_error: 5.6258

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━

2024/10/23 00:39:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run adventurous-dog-953 at:  http://127.0.0.1:5000/#/experiments/512988094738068919/runs/a27542434df946e8a68a2024666d73bb.

2024/10/23 00:39:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at:  http://127.0.0.1:5000/#/experiments/512988094738068919.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 20s 465ms/step - loss: 39.1877 - root_mean_squared_error: 6.2600
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.3564 - root_mean_squared_error: 6.0250
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.1470 - root_mean_squared_error: 5.8341
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.1897 - root_mean_squared_error: 5.4826
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.3227 - root_mean_squared_error: 5.4826
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.6588 - root_mean_squared_error: 5.3205
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27.1682 - root_mean_squared_error: 5.1703   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 10.0781 - root_mean_squared_error: 2.9991 - val_loss: 1.2976 - val_root_mean_squared_error: 1.1391

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.4125 - root_

2024/10/23 00:39:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run brawny-turtle-425 at:  http://127.0.0.1:5000/#/experiments/512988094738068919/runs/fa38042e146145dea04ca5e03c105d70.

2024/10/23 00:39:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at:  http://127.0.0.1:5000/#/experiments/512988094738068919.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 12s 271ms/step - loss: 37.6005 - root_mean_squared_error: 6.1319
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.6632 - root_mean_squared_error: 6.1370
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.9373 - root_mean_squared_error: 6.1592
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.9808 - root_mean_squared_error: 6.1628
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.9729 - root_mean_squared_error: 6.1622   
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37.1776 - root_mean_squared_error: 6.0970 - val_loss: 33.5609 - val_root_mean_squared_error: 5.7932

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 33.9010 - root_mean_squared_error: 5.8389
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - loss: 34.0935 - root_mean_squared_error: 5.8179
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - loss: 33.9159 -

2024/10/23 00:40:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run dapper-turtle-383 at:  http://127.0.0.1:5000/#/experiments/512988094738068919/runs/5b63ab60fc4d4099a2144dff2d21e9c6.

2024/10/23 00:40:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at:  http://127.0.0.1:5000/#/experiments/512988094738068919.



100%|██████████| 8/8 [00:55<00:00,  6.90s/trial, best loss: 0.8738760352134705]


2024/10/23 00:40:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run able-pig-921 at:  http://127.0.0.1:5000/#/experiments/512988094738068919/runs/fb88bd31aeb846a88c790af0a3eb7da0.
2024/10/23 00:40:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at:  http://127.0.0.1:5000/#/experiments/512988094738068919.


Best parameters: {'lr': 0.01051513745776196, 'momentum': 0.3995259729788678}
Best eval rmse: 0.8738760352134705


In [15]:
## Inferece
from mlflow.models import validate_serving_input

model_uri = 'runs:/fb88bd31aeb846a88c790af0a3eb7da0/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)
# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[5.165843 ],
       [7.076086 ],
       [6.3972936],
       ...,
       [6.486343 ],
       [6.6715965],
       [5.423612 ]], dtype=float32)